In [8]:
!pip uninstall transformers

^C


In [158]:
import nltk
nltk.download('punkt')  # Download the punkt tokenizer data if not already downloaded

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\EB34710\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [82]:
from transformers import pipeline
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first") # pass device=0 if using gpu

In [83]:
def extract_entities_from_note(note_text):
    data = pipe(note_text)
    data.sort(key=lambda x: x['start'])


    combined_data = []
    current_dict = data[0]

    for dict_item in data[1:]:
        if dict_item['start'] == current_dict['end'] + 1:
            current_dict['end'] = dict_item['end']
            current_dict['word'] += ' ' + dict_item['word']
            current_dict['entity_group'] += ' '+ dict_item['entity_group']
        else:
            combined_data.append(current_dict)
            current_dict = dict_item

    # Append the last dictionary
    combined_data.append(current_dict)

    return [(dic['word'],dic['start'],dic['end'],dic['entity_group']) for dic in combined_data]


In [ ]:
filter_list = ['Activity','Administration','Area','Biological_attribute','Biological_structure','Clinical_event','Coreference','Detailed_description','Diagnostic_procedure','Disease_disorder','Family_history','History','Mass','Medication','Non[biological](Detailed_description','Nonbiological_location','Other_entity','Other_event','Outcome','Personal_[back](Biological_structure','Personal_background','Qualitative_concept','Quantitative_concept','Sign_symptom','Texture','Therapeutic_procedure']
def filter_clinical_entities(entities_tuples_list):
    entities_tuples_list_filtered = []
    for entity_tuple in entities_tuples_list:
        entity_type = entity_tuple[3]
        for filter_val in filter_list:
            if filter_val in entity_type:
                entities_tuples_list_filtered.append((entity_tuple))
                break
    return entities_tuples_list_filtered

In [85]:
df = pd.read_csv(r"C:\Users\EB34710\Downloads\mimic-iv_notes_training_set.csv")
df['entites_extracted'] = df['text'].apply(lambda x: extract_entities_from_note(x))
df['clinical_entites_extracted'] = df['entites_extracted'].apply(lambda x:filter_clinical_entities(x) )

In [159]:
df.head(1)

,note_id,text,entites_extracted,clinical_entites_extracted
0,10060142-DS-9,\nName: ___ Unit No: ___\...,"[(penicillins, 179, 190, Medication), (biliary...","[(penicillins, 179, 190, Medication), (biliary..."


In [171]:
df['both_sided_bi_grams'] = df.apply(lambda x: helper(x['text'],x['clinical_entites_extracted']), axis=1)

ValueError: 'pneumonia' is not in list

In [170]:
def helper(text, entity_tuples_list):
    bigrams_list  = []
    for entity_tuple in entity_tuples_list:
        s_i = entity_tuple[1]
        e_i = entity_tuple[2]
        bigrams_list.append(generate_n_gram_main(text,s_i,e_i))
    return bigrams_list

In [160]:
def find_sentence(paragraph, start_index, end_index):
    # Tokenize the paragraph into sentences
    sentences = nltk.sent_tokenize(paragraph)

    # Iterate through each sentence to find the one containing the specified substring
    for i, sentence in enumerate(sentences):
        sentence_start = paragraph.find(sentence)
        sentence_end = sentence_start + len(sentence)

        # Check if the specified substring is within the current sentence
        if start_index >= sentence_start and end_index <= sentence_end:
            return sentence

    # If no matching sentence is found, return None
    return None

In [168]:
### -2 +2 ###
def generate_n_gram(reqd_sent_tokens, start_token_index,end_token_index,left_tokens,right_tokens):
    if start_token_index-left_tokens>=0 and end_token_index+right_tokens+1  <=len(reqd_sent_tokens):
        token_range = reqd_sent_tokens[start_token_index-left_tokens : end_token_index+right_tokens+1]
        n_gram = ' '.join(token_range)
        return (n_gram, start_token_index-left_tokens, end_token_index+right_tokens+1)
    return "None"

In [169]:
start_index = 179
end_index = 190
def generate_n_gram_main(text, start_index,end_index):
    reqd_sent =  find_sentence(text, start_index, end_index)
    reqd_sent_tokens = nltk.word_tokenize(reqd_sent)

    substring = text[start_index:end_index]
    substring_tokens_list = nltk.word_tokenize(substring)
    
    start_token = substring_tokens_list[0]
    end_token = substring_tokens_list[-1]

    start_token_index= reqd_sent_tokens.index(start_token)
    end_token_index = reqd_sent_tokens.index(end_token)

    return generate_n_gram(reqd_sent_tokens, start_token_index, end_token_index,2,2)


In [167]:
generate_n_gram_main(text,start_index,end_index)

'Allergies : Penicillins Attending :'

In [156]:
generate_n_gram(start_token_index,end_token_index, 1,2)

': Penicillins Attending :'

In [82]:
df.to_csv('extracted_entities_from_clinical_notes.csv',index=False)

In [59]:
df['text'] = df['text'].apply(lambda x:x.strip('\r'))

In [60]:
df_annotated = pd.read_csv("data\snomed-ct-entity-challenge\\1.0.0\\train_annotations.csv")

In [61]:
df_annotated['entities_annotated'] = df_annotated.apply(lambda x: (x['start'],x['end']), axis=1)

In [63]:
df_annotated

,note_id,start,end,concept_id,entities_annotated
0,10060142-DS-9,179,190,91936005,"(179, 190)"
1,10060142-DS-9,228,248,95563007,"(228, 248)"
2,10060142-DS-9,294,322,45595009,"(294, 322)"
3,10060142-DS-9,390,411,95563007,"(390, 411)"
4,10060142-DS-9,425,444,1835003,"(425, 444)"
...,...,...,...,...,...
51569,19926965-DS-14,9216,9227,76948002,"(9216, 9227)"
51570,19926965-DS-14,9257,9261,22253000,"(9257, 9261)"
51571,19926965-DS-14,9298,9302,22253000,"(9298, 9302)"
51572,19926965-DS-14,9318,9323,386661006,"(9318, 9323)"


In [64]:
df_annotated = df_annotated.groupby('note_id').agg({'entities_annotated':list}).reset_index()

In [65]:
df_annotated.shape

(204, 2)

In [66]:
df.shape

(204, 2)

In [67]:
df = pd.merge(df,df_annotated, on='note_id',how='inner')

In [68]:
def get_entity_from_indices(text, entities_annotated_list):
    entities_list =[]
    for entity_tuple in entities_annotated_list:
        entities_list.append((text[ entity_tuple[0]:entity_tuple[1]],entity_tuple[0],entity_tuple[1]))

In [86]:
pipe(df['text'].iloc[0])

[{'entity_group': 'Medication',
  'score': 0.780673,
  'word': 'penicillins',
  'start': 189,
  'end': 200},
 {'entity_group': 'Biological_structure',
  'score': 0.9990392,
  'word': 'biliary',
  'start': 243,
  'end': 250},
 {'entity_group': 'Disease_disorder',
  'score': 0.9887317,
  'word': 'pancreatitis',
  'start': 251,
  'end': 263},
 {'entity_group': 'Detailed_description',
  'score': 0.7746432,
  'word': 'major',
  'start': 268,
  'end': 273},
 {'entity_group': 'Therapeutic_procedure',
  'score': 0.83981717,
  'word': 'cholecystectomy',
  'start': 325,
  'end': 340},
 {'entity_group': 'Sex',
  'score': 0.9921673,
  'word': 'man',
  'start': 393,
  'end': 396},
 {'entity_group': 'Severity',
  'score': 0.99935836,
  'word': 'severe',
  'start': 405,
  'end': 411},
 {'entity_group': 'Biological_structure',
  'score': 0.9998154,
  'word': 'biliary',
  'start': 412,
  'end': 419},
 {'entity_group': 'Disease_disorder',
  'score': 0.9916431,
  'word': 'pancreatitis',
  'start': 422,
 

In [88]:
pipe('Patient referred for a biopsy to investigate potential swelling in upper larynx')

[{'entity_group': 'Diagnostic_procedure',
  'score': 0.99994206,
  'word': 'biopsy',
  'start': 23,
  'end': 29},
 {'entity_group': 'Sign_symptom',
  'score': 0.9999409,
  'word': 'swelling',
  'start': 55,
  'end': 63},
 {'entity_group': 'Biological_structure',
  'score': 0.9998735,
  'word': 'upper larynx',
  'start': 67,
  'end': 79}]

In [91]:
pipe('pain in the hands')

[{'entity_group': 'Sign_symptom',
  'score': 0.999943,
  'word': 'pain',
  'start': 0,
  'end': 4},
 {'entity_group': 'Biological_structure',
  'score': 0.9939996,
  'word': 'hands',
  'start': 12,
  'end': 17}]

In [69]:
print(df.text.iloc[0][179:190])

ergies: 
P


In [34]:
df['entities_annotated'] = df.apply(lambda x:get_entity_from_indices(x['text'],x['entities_annotated']),axis=1)

In [39]:
df['entites_extracted'].iloc[0]

[('penicillins', 189, 200),
 ('biliary pancreatitis', 243, 263),
 ('major', 268, 273),
 ('cholecystectomy', 325, 340),
 ('man', 393, 396),
 ('severe biliary', 405, 419),
 ('pancreatitis', 422, 434),
 ('pancreatic necrosis', 448, 467),
 ('multisystem', 566, 577),
 ('postnecrotic pseudocyst', 631, 654),
 ('drained', 673, 680),
 ('minimally invasive approach', 691, 718),
 ('gi', 728, 730),
 ('cholecystectomy', 985, 1000),
 ('esophagus', 1057, 1066),
 ('esophageal ulcer', 1070, 1086),
 ('anxiety', 1090, 1097),
 ('pain', 1106, 1110),
 ('mother', 1155, 1161),
 ('laparoscopic incisions open to air', 1387, 1421),
 ('no', 1465, 1467),
 ('gallstone pancreatitis', 1573, 1595),
 ('admitted', 1600, 1608),
 ('laparoscopic cholecystectomy', 1667, 1695),
 ('arrived', 1755, 1762),
 ('dilaudid pca', 1822, 1834),
 ('pain', 1897, 1901),
 ('subcutaneous', 2007, 2019),
 ('heparin', 2022, 2029),
 ('venodyne boots', 2034, 2048),
 ('stay', 2071, 2075)]

In [43]:
df['entities_annotated'].iloc[0]

[('ergies: \r\nP', 179, 190),
 ('ef Complaint:\r\nBilia', 228, 248),
 (' Procedure:\r\n___: Laparoscop', 294, 322),
 ('__ man who had severe', 390, 411),
 ('creatitis resulting', 425, 444),
 ('h he was \r\ntreated w', 476, 496),
 ('asojejunal feed', 501, 516),
 ('  He had \r\ninitially had ', 541, 566),
 ('tem orga', 574, 582),
 (' has a lar', 618, 628),
 ('eudocys', 646, 653),
 ('invasive', 701, 709),
 ('lowed ', 818, 824),
 ('n his ___ \r\ncli', 836, 851),
 ('iscuss cho', 859, 869),
 ('nt was schedule', 953, 968),
 ('on ___.\r\n \r\nP', 1001, 1014),
 ('ical Hist', 1021, 1030),
 (':\r\nHiatal Hernia', 1033, 1049),
 ('\n___ es', 1052, 1059),
 ('agus  \r\nE', 1062, 1071),
 ('ry:\r\n___\r\nFamily Histo', 1128, 1150),
 ('er pa', 1159, 1164),
 ('f m', 1170, 1173),
 ('her alive, had CVA \r\n', 1199, 1220),
 ('ig', 1258, 1260),
 ('\r\n', 1288, 1290),
 ('rio', 1291, 1294),
 ('Dis', 1296, 1299),
 ('S:', 1309, 1311),
 ('8.3', 1313, 1316),
 (' 83,', 1317, 1321),
 ('37/6', 1323, 1327),
 (', 1', 1

In [76]:
df.text.iloc[0].strip(' \r')[179:190]

'gies: \r\nPen'